# PIE Air France: Robustesse chaînage avion

# Mise en forme des données

In [1]:
#Les import

import pandas
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from math import isnan
import seaborn as sns
from sklearn.decomposition import PCA


In [2]:
vols_MC = pandas.read_csv("./Data/vols_MC_cleaned.csv",sep=',',parse_dates=['DDP','DDR','DAP','DAR'],decimal=b'.')

In [55]:
vols_MC.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Identifiant vol', 'TTM HUB',
       'Type avion réalisé', 'Code retard', 'Temps de vol bloc bloc prévu',
       'Temps de vol bloc bloc réalisé', 'Immatriculation',
       'Type exploitation', 'Escale départ réalisée',
       'Escale arrivée réalisée', 'Nb de pax embarqués à l'escale',
       'Date départ prévue TU', 'Heure départ prévue TU',
       'Date départ réalisée TU', 'Heure départ réalisée TU',
       'Date arrivée prévue TU', 'Date arrivée réalisée TU',
       'Heure arrivée réalisée TU', 'Heure arrivée prévue TU', 'DDP', 'DDR',
       'DAP', 'DAR', 'Retard_D', 'Retard_A', 'Week_day', 'day_index',
       'Cat_retard'],
      dtype='object')

In [56]:
Imm = 'FGUGB'
chainage_avion = vols_MC[vols_MC['Immatriculation']==Imm]
chainage_avion = chainage_avion.sort_values('DDR')[['Date départ réalisée TU','Escale départ réalisée','Escale arrivée réalisée','DDR','Retard_D']]
chainage_avion = chainage_avion.assign(Retard_rélatif = 0)
chainage_avion['Retard_D'].values[0]
for i in range(chainage_avion.shape[0]):
    if i == 0:
        chainage_avion['Retard_rélatif'].values[i]= chainage_avion['Retard_D'].values[i]
    else:
        if chainage_avion['Date départ réalisée TU'].values[i-1] == chainage_avion['Date départ réalisée TU'].values[i]:
            chainage_avion['Retard_rélatif'].values[i]= chainage_avion['Retard_D'].values[i] - chainage_avion['Retard_D'].values[i-1]
        else:
            chainage_avion['Retard_rélatif'].values[i]= chainage_avion['Retard_D'].values[i]
chainage_avion.head()

,Date départ réalisée TU,Escale départ réalisée,Escale arrivée réalisée,DDR,Retard_D,Retard_rélatif
121,2009/07/01,LYS,NTE,2009-07-01 04:15:00,-5.0,-5
69,2009/07/01,NTE,LYS,2009-07-01 06:16:00,-4.0,1
34,2009/07/01,LYS,TUN,2009-07-01 09:47:00,12.0,16
512,2009/07/01,TUN,LYS,2009-07-01 12:19:00,-1.0,-13
328,2009/07/01,LYS,CDG,2009-07-01 15:02:00,7.0,8


In [39]:
list(chainage_avion[chainage_avion['Date départ réalisée TU']=='2009/07/01']['Escale départ réalisée'])[0]

'LYS'

In [28]:
Liste_date = list(chainage_avion.groupby(['Date départ réalisée TU'])[['Retard_rélatif']].groups.keys())
len(Liste_date)

63

In [62]:
chaiange_jour_avion = pandas.DataFrame()
chaiange_jour_avion = chaiange_jour_avion.assign(Date = Liste_date)
chaiange_jour_avion = chaiange_jour_avion.assign(Imm = 'FGUGB')


Colonne=[]
for d in Liste_date:
    L=''
    Liste_air_dep = list(chainage_avion[chainage_avion['Date départ réalisée TU']==d]['Escale départ réalisée'])
    Liste_air_air = list(chainage_avion[chainage_avion['Date départ réalisée TU']==d]['Escale arrivée réalisée'])
    for air in Liste_air_dep:
        L=L+'-'+air
    L=L+'-'+ Liste_air_air[-1]   
    Colonne.append(L[1:])
chaiange_jour_avion = chaiange_jour_avion.assign(Chainage = Colonne)
chaiange_jour_avion

,Date,Imm,Chainage
0,2009/07/01,FGUGB,LYS-NTE-LYS-TUN-LYS-CDG-OSL
1,2009/07/02,FGUGB,OSL-CDG-TRN-CDG-WAW-CDG
2,2009/07/03,FGUGB,CDG-FRA-CDG-BLQ-CDG-DUS-CDG
3,2009/07/04,FGUGB,CDG-FRA-CDG-BLQ-CDG-PUF
4,2009/07/05,FGUGB,PUF-CDG-TRN-CDG-TRN-CDG-TLS
5,2009/07/06,FGUGB,TLS-CDG
6,2009/07/07,FGUGB,CDG-PUF-CDG-PUF-CDG
7,2009/07/08,FGUGB,CDG-MUC-CDG-MAD-CDG-LYS
8,2009/07/09,FGUGB,LYS-TLS-LYS-CDG-CPH-CDG-DUS-CDG
9,2009/07/10,FGUGB,CDG-CPH-CDG-FRA-CDG-BES


In [61]:
chaiange_jour_avion.groupby(['Chainage']).count()

,Date,Imm
Chainage,,
BES-CDG-PUF-CDG-DUS-CDG-FRA,1,1
BES-CDG-PUF-CDG-FRA-CDG-DUS-CDG,1,1
BES-CDG-PUF-CDG-FRA-CDG-MAN-CDG,1,1
BES-CDG-PUF-CDG-MAN-CDG,2,2
BLQ-CDG-FRA-CDG,1,1
BLQ-CDG-FRA-CDG-AMS-CDG,1,1
BLQ-CDG-FRA-CDG-FRA-CDG-BLQ,1,1
BLQ-CDG-MUC-CDG-TRN-CDG-BES,1,1
BUD-CDG-GVA-CDG-AMS-CDG,1,1
